In [204]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture

In [205]:
fisher_vectors = np.zeros((80*3,256*(2*80+1)))

In [206]:
for i in range(0,3):
    for j in range(80):
        file_name = 'Descriptors_compressed/descriptors_reduced'
        file_name += str(80*i + j)
        file_name += '.csv'
        
        try:
            df = pd.read_csv(file_name)
        except FileNotFoundError:
            print('File does not exist')
        else:
            df = pd.read_csv(file_name)
            total_descriptors = df.shape[0]
            X = df.to_numpy()
            GMM = GaussianMixture(n_components=256, covariance_type='diag').fit(X)
            print('Iteration: ', 80*i + j)
            print('Converged: ', GMM.converged_)
            means = GMM.means_ 
            var_square = GMM.covariances_
            var = np.sqrt(var_square)
            weights = GMM.weights_
            prediction = GMM.predict_proba(X)
        
            fisher_vector = np.zeros(shape=(256, 2*80+1))
            for k in range(256):
                alpha_k = 0
                mi_k = np.zeros(80)
                sigma_k = np.zeros(80)
            
                for t in range(total_descriptors):
                    alpha_k += (prediction[t][k] - weights[k])
                    mi_k += prediction[t][k]*((X[t] - means[k])/var[k])
                    sigma_k += (prediction[t][k]/np.sqrt(2))*((((X[t] - means[k])*(X[t] - means[k]))/var_square[k]) - np.ones(80))
            
                alpha_k /= np.sqrt(weights[k])
                mi_k /= np.sqrt(weights[k])
                sigma_k /= np.sqrt(weights[k])
            
                fisher_vector[k][0] = alpha_k

                for d in range(80):
                    fisher_vector[k][d+1] = mi_k[d]
                    fisher_vector[k][d+81] = sigma_k[d]
        
            fisher_vector = fisher_vector.reshape(256*(2*80+1))
            fisher_vector /= total_descriptors
            power_normalization = lambda t: np.sign(t)*np.sqrt(np.abs(t))
            vfunc = np.vectorize(power_normalization)
            vfunc(fisher_vector)
            fisher_vector /= np.sqrt((fisher_vector**2).sum())
            fisher_vectors[80*i+j] = fisher_vector

Iteration:  0
Converged:  True


KeyboardInterrupt: 

In [ ]:
fisher_vectors.shape

In [ ]:
colnames.clear()

for i in range(256*(2*80+1)):
    key = 'V' + str(i+1)
    colnames.append(key)

In [173]:
df = pd.DataFrame(data=fisher_vectors[0:,0:], columns=colnames)

In [177]:
df.to_csv('FV/FV0_3.csv', index=False)